In [1]:
import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, ShortType, DoubleType, DateType
from pyspark.sql import functions as F

In [2]:
try:
    spark.stop()
except:
    print('spark not initialized')

spark not initialized


In [3]:
spark = SparkSession \
    .builder \
    .master('spark://master:7077') \
    .appName("Sparkify Local ETL") \
    .getOrCreate()

spark.newSession()

In [4]:
# /opt/data/song_data/
# └── A
#     ├── A
#     │   ├── A
#     │   │   ├── TRAAAAK128F9318786.json
#     │   │   ├── TRAAAAV128F421A322.json
#     │   │   ├── TRAAABD128F429CF47.json
#     │   │   ├── TRAAACN128F9355673.json

song_schema = StructType([
    StructField("artist_id", StringType(), True),
    StructField("artist_name", StringType(), True),
    StructField("artist_location", StringType(), True),
    StructField("artist_latitude", DoubleType(), True),
    StructField("artist_longitude", DoubleType(), True),
    StructField("song_id", StringType(), True),
    StructField("duration", DoubleType(), True),
    StructField("title", StringType(), True),
    StructField("year", ShortType(), True),
])

# song_df = spark.read.json('/opt/data/song_data/A/A/A/*.json', song_schema).cache()
song_df = spark.read.json('/opt/data/song_data/*/*/*/*.json', song_schema).cache()
song_df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- song_id: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year: short (nullable = true)



In [5]:
print('song record count:', song_df.count())

song record count: 14896


In [8]:
song_df.limit(3).toPandas()

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude,song_id,duration,title,year
0,AR4T2IF1187B9ADBB7,Billy Idol,"<a href=""http://billyidol.net"" onmousedown='Un...",63.96027,10.22442,SOVIYJY12AF72A4B00,233.22077,The Dead Next Door (Digitally Remastered 99),1983
1,AR4T2IF1187B9ADBB7,Billy Idol,"<a href=""http://billyidol.net"" onmousedown='Un...",63.96027,10.22442,SOVYXYL12AF72A3373,287.92118,Rebel Yell (1999 Digital Remaster),1983
2,ARQ846I1187B9A7083,Yvonne S. Moriarty / Walt Fowler / Ladd McInto...,,NaN,NaN,SOEPTVC12A67ADD0DA,196.04853,"To Zucchabar [""Gladiator"" - Music from the Mot...",0


In [9]:
# workspace/data/log_data/
# └── 2018
#     └── 11
#         ├── 2018-11-01-events.json
#

event_df = spark.read.json('/opt/data/log_data/*/*/*.json') \
    .filter(F.col('page') == 'NextSong') \
    .withColumn('timestamp', F.from_unixtime(F.col('ts') / 1000)) \
    .withColumn('start_time', F.date_format('timestamp', 'yyyyMMddHH')) \
    .cache()

event_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- start_time: string (nullable = true)



In [10]:
print('event (log) record count:', event_df.count())

event (log) record count: 6820


In [11]:
event_df.limit(3).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,timestamp,start_time
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:30:26,2018111500
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:41:21,2018111500
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 00:45:41,2018111500


# Start Transform

## User Dimension DataFrame

In [12]:
d_user_df = event_df.select('userId', 'lastName', 'firstName', 'gender') \
    .dropDuplicates(['userId']) \
    .cache()


In [13]:
print('user dimension record count: ', d_user_df.count())

user dimension record count:  96


In [14]:
d_user_df.limit(3).toPandas()

,userId,lastName,firstName,gender
0,51,Burke,Maia,F
1,7,Jordan,Adelyn,F
2,15,Koch,Lily,F


## Artist Dimension DataFrame

In [15]:
d_artist_df = song_df.select('artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude') \
    .dropDuplicates(['artist_id']) \
    .cache()

In [16]:
print('artist dimension record count:', d_artist_df.count())

artist dimension record count: 9553


In [17]:
d_artist_df.limit(3).toPandas()

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR0G85S1187FB4D46D,Hey Hey My My,,NaN,NaN
1,AR13CXU1187B9AD30A,The Clash,"London, England",51.52328,-0.21346
2,AR1X3SU11C8A415261,Vessels,Hanover Park/Schaumburg,41.99112,-88.15849


## Song Dimension DataFrame

In [18]:
d_song_df = song_df.select('song_id', 'title', 'artist_id', 'year', 'duration') \
    .cache()

In [19]:
print('song dimension record count:', d_song_df.count())

song dimension record count: 14896


In [20]:
d_song_df.limit(3).toPandas()

,song_id,title,artist_id,year,duration
0,SOVIYJY12AF72A4B00,The Dead Next Door (Digitally Remastered 99),AR4T2IF1187B9ADBB7,1983,233.22077
1,SOVYXYL12AF72A3373,Rebel Yell (1999 Digital Remaster),AR4T2IF1187B9ADBB7,1983,287.92118
2,SOEPTVC12A67ADD0DA,"To Zucchabar [""Gladiator"" - Music from the Mot...",ARQ846I1187B9A7083,0,196.04853


## Time Dimension DataFrame

In [21]:
d_time_df = event_df.select('timestamp', 'start_time') \
    .withColumn('year', F.year('timestamp')) \
    .withColumn('month', F.month('timestamp')) \
    .withColumn('day', F.dayofmonth('timestamp')) \
    .withColumn('hour', F.hour('timestamp')) \
    .withColumn('week_of_year', F.weekofyear('timestamp')) \
    .withColumn('weekday', F.dayofweek('timestamp')) \
    .select(['start_time', 'year', 'month', 'day', 'hour', 'week_of_year', 'weekday']) \
    .dropDuplicates(['start_time'])


In [22]:
print('time dimension record count:', d_time_df.count())

time dimension record count: 544


In [23]:
d_time_df.limit(3).toPandas()

,start_time,year,month,day,hour,week_of_year,weekday
0,2018111710,2018,11,17,10,46,7
1,2018111004,2018,11,10,4,45,7
2,2018112508,2018,11,25,8,47,1


## Songplay Fact DataFrame

In [102]:
tmp_df = d_song_df.withColumnRenamed('artist_id', 'song_artist_id')
tmp_df = tmp_df.join(d_artist_df, d_artist_df.artist_id == tmp_df.song_artist_id) \
    .select('song_id', 'title', 'duration', 'artist_id', 'artist_name')

# print('tmp record count:', tmp_df.count())
tmp_df.limit(3).toPandas()

,song_id,title,duration,artist_id,artist_name
0,SOVIYJY12AF72A4B00,The Dead Next Door (Digitally Remastered 99),233.22077,AR4T2IF1187B9ADBB7,Billy Idol
1,SOVYXYL12AF72A3373,Rebel Yell (1999 Digital Remaster),287.92118,AR4T2IF1187B9ADBB7,Billy Idol
2,SOEPTVC12A67ADD0DA,"To Zucchabar [""Gladiator"" - Music from the Mot...",196.04853,ARQ846I1187B9A7083,Yvonne S. Moriarty / Walt Fowler / Ladd McInto...


In [121]:
comparison = [event_df.song == tmp_df.title, event_df.length.cast(ShortType()) == tmp_df.duration.cast(ShortType())]

f_songplay_df = event_df.withColumn('songplay_id', F.sha1(F.concat_ws('|', 'timestamp', 'userId', 'song'))) \
    .join(tmp_df, comparison, 'left') \
    .select(['songplay_id', 'start_time', 'userId', 'level', 'song_id', 'artist_id', 'sessionId', 'location', 'userAgent'])


In [122]:
print('songplay fact record count:', f_songplay_df.count())

songplay fact record count: 6820


In [123]:
c = f_songplay_df.filter(F.col('song_id').isNotNull()).count()
print('songplay fact records with song_id value:', c)

songplay fact records with song_id value: 325


In [125]:
# f_songplay_df.write.save('f_songplay_df.json', format='json')
f_songplay_df.toPandas().to_csv('f_songplay_df.csv')
d_user_df.toPandas().to_csv('d_user_df.csv')
d_song_df.toPandas().to_csv('d_song_df.csv')
d_artist_df.toPandas().to_csv('d_artist_df.csv')
d_time_df.toPandas().to_csv('d_time_df.csv')